# 1. Simulation of the Magnet rule and some alternatives

# 2. Validation of the spatial queueing models in the context of Toronto

In [ ]:
import matplotlib
%matplotlib inline
import numpy as np
import random
import matplotlib.pyplot as plt
from datascience import *


%matplotlib inline

# configurations 
matplotlib.rc("figure", figsize=(8,8))
matplotlib.rc("axes", labelsize=16, titlesize=16)
matplotlib.rc("xtick", labelsize=14)
matplotlib.rc("ytick", labelsize=14)
matplotlib.rc("legend", fontsize=14)
matplotlib.rc("font", size=14)


In [ ]:
def generate_customer_pos(k):
    # random() returns the value in [0, 1)
    x = random.randint(0,k)
   
    y = random.randint(0,k)

    return x, y

In [ ]:
def magnet_rule(x_now, y_now, x_c, y_c):
       
    x_next, y_next = magnet_rule_type_1(x_now, y_now, x_c, y_c)
            
    if y_next < phi*k or y_next > k-phi*k: 
        y_next = y_now
    
    if x_next < phi*k or x_next > k-phi*k:
        x_next = x_now
 
                
    return x_next, y_next




def magnet_rule_type_1(x_now, y_now, x_c, y_c):
        '''The Magnet Rule for Type-1 Origin'''
        
        if (x_c > x_now+phi*k) and (y_c > y_now+phi*k):
            Region = 'S1'
            x_next = x_c-phi*k
            y_next = y_c-phi*k
            
        elif (x_c > x_now+phi*k) and (y_now-phi*k <= y_c <= y_now+phi*k):
            Region = 'S2'
            x_next = x_c-phi*k
            y_next = y_c

        elif (x_c > x_now+phi*k) and (y_c < y_now-phi*k):
            Region = 'S3'
            x_next = x_c-phi*k
            y_next = y_c+phi*k
            
        elif (x_now-phi*k <= x_c <= x_now+phi*k) and (y_c < y_now-phi*k):
            Region = 'S4'
            x_next = x_c
            y_next = y_c+phi*k
            
        elif (x_c < x_now-phi*k) and (y_c < y_now-phi*k):
            Region = 'S5'
            x_next = x_c+phi*k
            y_next = y_c+phi*k
            
        elif (x_c < x_now-phi*k) and (y_now-phi*k <= y_c <= y_now+phi*k):
            Region = 'S6'
            x_next = x_c+phi*k
            y_next = y_c
            
        elif (x_c < x_now-phi*k) and (y_c > y_now+phi*k):
            Region = 'S7'
            x_next = x_c+phi*k
            y_next = y_c-phi*k
            
        elif (x_now-phi*k <= x_c <= x_now+phi*k)  and (y_c > y_now+phi*k):
            Region = 'S8'
            x_next = x_c
            y_next = y_c-phi*k
            
        else:
            Region = 'S0'
            x_next = x_now 
            y_next = y_now
            
        #print(Region)
            
        return x_next, y_next
    

In [ ]:
def straight_ahead_rule(x_now, y_now, x_c, y_c):
    
    #upright coordinates:
    wo = x_now/np.sqrt(2) - y_now/np.sqrt(2)
    vo = x_now/np.sqrt(2) + y_now/np.sqrt(2)
    wc = x_c/np.sqrt(2) - y_c/np.sqrt(2)
    vc = x_c/np.sqrt(2) + y_c/np.sqrt(2)
      
    if abs(wo-wc)+abs(vo-vc)<= phi*k:   
        # already within radius, no need to move.
        x_next = x_now 
        y_next = y_now
    else:
        if abs(x_now-x_c) >= abs(y_now-y_c):
            if x_c > x_now:
                x_next = x_c - phi*k
            else:
                x_next = x_c + phi*k
            
            # z_next = lamb * z_o + (1-lamb) * z_c
            # lamb = (z_next - z_c)/(z_o - z_c)
            
            lamb = (x_next-x_c) / (x_now - x_c)
            y_next = lamb * y_now + (1-lamb) * y_c
            
        else:
            if y_c > y_now:
                y_next = y_c - phi*k
            else:
                y_next = y_c + phi*k
            
            # z_next = lamb * z_o + (1-lamb) * z_c
            # lamb = (z_next - z_c)/(z_o - z_c)
            
            lamb = (y_next-y_c) / (y_now - y_c)
            x_next = lamb * x_now + (1-lamb) * x_c
            
        
    return x_next, y_next

  

In [ ]:
def distance(x1,y1,x2,y2):
    """return the l1 distance between points"""
    w1 = x1/np.sqrt(2) - y1/np.sqrt(2)
    v1 = x1/np.sqrt(2) + y1/np.sqrt(2)
    w2 = x2/np.sqrt(2) - y2/np.sqrt(2)
    v2 = x1/np.sqrt(2) + y2/np.sqrt(2)
    
    return abs(w1-w2)+abs(v1-v2)

def boundary_inner_diamond(x,y):
    """return the list of points that form the boundary of the inner diamond centering at (x,y)"""
    space = round(k*phi)
    boundary = np.ndarray((4*2*space,2))
    for i in range(2*space):
        boundary[i] = (x-space, y-space+i) # Southwest boundary
    for i in range(2*space):
        boundary[i+2*space] = (x-space+i, y+space) # Northwest boundary
    for i in range(2*space):
        boundary[i+4*space] = (x+space, y+space-i)  # Northest boundary
    for i in range(2*space):
        boundary[i+6*space] = (x+space-i, y-space)  # Southest boundary
    
    return boundary

        
def points_interest_on_boundary(xo,yo,xc1,yc1):
    
    boundary = boundary_inner_diamond(xc1,yc1)
        
    # The candidate zd are those such that vd is in [vo, vc] and wd is in [wo, wc]
    points_interest = []
    
    sqrt2 = np.sqrt(2)

    wo = xo/sqrt2 - yo/sqrt2
    vo = xo/sqrt2 + yo/sqrt2
    wc1 = xc1/sqrt2 - yc1/sqrt2
    vc1 = xc1/sqrt2 + yc1/sqrt2
    
    eps=0.1
    for point in boundary:
        point_w = point[0]/sqrt2 - point[1]/sqrt2
        point_v = point[0]/sqrt2 + point[1]/sqrt2
        if (wo-eps<=point_w<=wc1+eps or wc1-eps<=point_w<=wo+eps) and \
            (vo-eps<=point_v<=vc1+eps or vc1-eps<=point_v<=vo+eps) and \
            (k*phi-eps<=point[0]<=k-k*phi+eps and k*phi-eps<=point[1]<=k-k*phi+eps):  # this line is optional, just to consider the inner diamond

            points_interest.append(point) 

    if len(points_interest) == 0:  # debug
        
        
        plt.plot([0,k/np.sqrt(2)], [0,k/np.sqrt(2)], 'grey')
        plt.plot([0,-k/np.sqrt(2)], [0,k/np.sqrt(2)], 'grey')
        plt.plot([-k/np.sqrt(2), 0], [k/np.sqrt(2), k*np.sqrt(2)], 'grey')
        plt.plot([0,k/np.sqrt(2)], [k*np.sqrt(2),k/np.sqrt(2)], 'grey')

        plt.grid(False)
        plt.ylim([-200, 200])
        plt.xlim([-200, 200])
    
        plt.plot(wo, vo, 'dy')
        plt.plot((wo,wc1), (vo, vc1))
        plt.plot(wc1, vc1, '+r')
        plt.plot(point_w, point_v, 'ob', markersize = 30)

        for dot in boundary:
            plt.plot(dot[0]/sqrt2-dot[1]/sqrt2, dot[0]/sqrt2+dot[1]/sqrt2, 'd')

        print('Num. of points on the boundary: ', len(boundary))
        #print(boundary)
    
    return points_interest



def next_two_redezvous(xo,yo,xc1,yc1,xc2,yc2):
    """return the next rendezvous that minimize the sum of the next two repo trip lengths"""
    
    xd1_temp, yd1_temp = magnet_rule_type_1(xo, yo, xc1, yc1)
    dist_od1 = distance(xo,yo,xd1_temp,yd1_temp) # this is the shortest and optimal distance for the next repo trip.
     
    if dist_od1 < 0.1:  # zd = zo
        xd1_opt = xo
        yd1_opt = yo
        total_distance_opt = distance(xo,yo,xc2,yc2)
    else: 
        points_interest = points_interest_on_boundary(xo,yo,xc1,yc1)
                
        dist_d1d2_opt = 999999999
        xd1_opt, yd1_opt = -100, -100
        
        
        for zd in points_interest:
            xd2_temp, yd2_temp = magnet_rule_type_1(zd[0], zd[1], xc2, yc2)
            dist_d1d2 = distance(zd[0],zd[1], xd2_temp, yd2_temp)
            if dist_d1d2 < dist_d1d2_opt:
                dist_d1d2_opt = dist_d1d2
                xd1_opt, yd1_opt = zd[0],zd[1] 
        
        total_distance_opt = dist_od1 + dist_d1d2
                    
    return xd1_opt,yd1_opt, total_distance_opt

In [ ]:
def next_three_redezvous(xo,yo,xc1,yc1,xc2,yc2,xc3,yc3):
    """return the next rendezvous that minimize the sum of the next THREE repo trip lengths"""
    
    xd1_temp, yd1_temp = magnet_rule_type_1(xo, yo, xc1, yc1)
    dist_od1 = distance(xo,yo,xd1_temp,yd1_temp) # this is the shortest and optimal distance for the next repo trip.
     
    if dist_od1 == 0:  # zd = zo
        xd1_opt = xo
        yd1_opt = yo
        total_dist_opt = next_two_redezvous(xo,yo,xc2,yc2,xc3,yc3)
        
    else: 
        points_interest = points_interest_on_boundary(xo,yo,xc1,yc1)
       
        total_dist_opt = 8888888
        for zd in points_interest:
            xd2_temp,yd2_temp, distance_d1d3 = next_two_redezvous(zd[0],zd[1],xc2,yc2,xc3,yc3)
            
            total_dist = dist_od1+distance_d1d3
            
            if total_dist < total_dist_opt:
                total_dist_opt = total_dist
                xd1_opt, yd1_opt = zd[0],zd[1] 
                    
    return xd1_opt,yd1_opt, total_dist_opt

In [ ]:
R = 100
k=R

N = 10

mean_repo = -88*np.ones(N)
mean_repo_alt = -88*np.ones(N)
mean_repo_d2d = -88*np.ones(N)
mean_repo_two_opt = -88*np.ones(N)
mean_repo_three_opt = -88*np.ones(N)

rr = (np.arange(0,5*N,5)+5)*np.sqrt(2)
phi_array = rr/np.sqrt(2)/R

for j in range(N):
#for j in [1]:        
    r = rr[j]
    phi = r/np.sqrt(2)/R   # phi is from 0 to 1/2/ 
    
    print('Round: ', j, '\t', 'phi: ', phi, '\n')


    num_trials = 5000

    x_c = R/2*np.ones(num_trials)
    y_c = R/2*np.ones(num_trials)

    x_array = R/2*np.ones(num_trials)
    y_array = R/2*np.ones(num_trials)

    x_array_alt = R/2*np.ones(num_trials)
    y_array_alt = R/2*np.ones(num_trials)

    x_array_two_opt = R/2*np.ones(num_trials)
    y_array_two_opt = R/2*np.ones(num_trials)

    x_array_three_opt = R/2*np.ones(num_trials)
    y_array_three_opt = R/2*np.ones(num_trials)


    for i in range(num_trials):
            #  customer's location
        x_c[i], y_c[i] = generate_customer_pos(k)

    for i in range(num_trials-1):    
        x_array[i+1], y_array[i+1] = magnet_rule(x_array[i], y_array[i], x_c[i+1], y_c[i+1] )

    for i in range(num_trials-1):    
        x_array_alt[i+1], y_array_alt[i+1] = straight_ahead_rule(x_array_alt[i], y_array_alt[i], x_c[i+1], y_c[i+1] )

    for i in range(num_trials-2):    
        x_array_two_opt[i+1], y_array_two_opt[i+1], total_dist = next_two_redezvous(x_array_two_opt[i], y_array_two_opt[i], 
                                                                        x_c[i+1], y_c[i+1], x_c[i+2], y_c[i+2])
    for i in range(num_trials-3):    
        x_array_three_opt[i+1], y_array_three_opt[i+1], total_dist = next_three_redezvous(x_array_three_opt[i], y_array_three_opt[i], 
                                                                        x_c[i+1], y_c[i+1], x_c[i+2], y_c[i+2], x_c[i+3], y_c[i+3])


    # change the coordinate from x-y to v-w to be upright:
    w_array = x_array/np.sqrt(2) - y_array/np.sqrt(2)
    v_array = x_array/np.sqrt(2) + y_array/np.sqrt(2)

    w_array_alt = x_array_alt/np.sqrt(2) - y_array_alt/np.sqrt(2)
    v_array_alt = x_array_alt/np.sqrt(2) + y_array_alt/np.sqrt(2)

    wc_array = x_c/np.sqrt(2) - y_c/np.sqrt(2)
    vc_array = x_c/np.sqrt(2) + y_c/np.sqrt(2)

    w_array_two_opt = x_array_two_opt/np.sqrt(2) - y_array_two_opt/np.sqrt(2)
    v_array_two_opt = x_array_two_opt/np.sqrt(2) + y_array_two_opt/np.sqrt(2)

    w_array_three_opt = x_array_three_opt/np.sqrt(2) - y_array_three_opt/np.sqrt(2)
    v_array_three_opt = x_array_three_opt/np.sqrt(2) + y_array_three_opt/np.sqrt(2)


    mean_repo[j] = np.mean(np.abs(w_array[2:]-w_array[1:-1]) + np.abs(v_array[2:]-v_array[1:-1]) )
    mean_repo_alt[j] = np.mean(np.abs(w_array_alt[2:]-w_array_alt[1:-1]) + np.abs(v_array_alt[2:]-v_array_alt[1:-1]) )

    mean_repo_d2d[j] = np.mean(np.abs(wc_array[2:]-wc_array[1:-1]) + np.abs(vc_array[2:]-vc_array[1:-1]) )

    mean_repo_two_opt[j] = np.mean(np.abs(w_array_two_opt[2:-1]-w_array_two_opt[1:-2]) + np.abs(v_array_two_opt[2:-1]-v_array_two_opt[1:-2]) )

    mean_repo_three_opt[j] = np.mean(np.abs(w_array_three_opt[2:-2]-w_array_three_opt[1:-3]) + np.abs(v_array_three_opt[2:-2]-v_array_three_opt[1:-3]) )


    print('Mean Repo Distance (Magnet Rule): ', mean_repo[j], '\n', 'Mean Repo Distance (Straight ahead rule): ', 
          mean_repo_alt[j], '\n', 'Mean Repo Distance D2D: ', mean_repo_d2d[j], '\n',
         'Mean Repo Distance (Two Opt): ', mean_repo_two_opt[j], '\n',
          'Mean Repo Distance (Three Opt): ', mean_repo_three_opt[j], '\n',
         'Relative Gap Between Magnet Rule and Two-Opt Rule: ', (mean_repo[j]-mean_repo_two_opt[j])/mean_repo[j], '\n',
         'Relative Gap Between Two-Opt Rule and Three-Opt Rule: ', (mean_repo_two_opt[j]-mean_repo_three_opt[j])/mean_repo_two_opt[j])



In [ ]:
plt.plot(phi_array, mean_repo, label='Magnet Rule')
plt.plot(phi_array, mean_repo_alt, label='Straight Ahead')
plt.plot(phi_array, mean_repo_d2d, label='D2D')
plt.plot(phi_array, mean_repo_two_opt, label='Two Opt')
plt.plot(phi_array, mean_repo_three_opt, label='Three Opt')
plt.legend()
plt.grid(True)


        

In [ ]:
Result_Summary = Table().with_columns('r/R', phi_array*np.sqrt(2),
                                    'Magnet Rule',mean_repo,
                                    'Straight Ahead',mean_repo_alt,
                                    'D2D',mean_repo_d2d,
                                    'Two Opt',mean_repo_two_opt,
                                    'Three Opt',mean_repo_three_opt,
                                   )

Result_Summary.to_csv('Mean_Repo_Summary.csv')

## Heatmaps 

In [ ]:
def plotting(ws,vs,bin_size, title='None'):
    fig, ax = plt.subplots()
    #ax.axis("off")

    plt.plot([0,k/np.sqrt(2)], [0,k/np.sqrt(2)], 'grey')
    plt.plot([0,-k/np.sqrt(2)], [0,k/np.sqrt(2)], 'grey')
    plt.plot([-k/np.sqrt(2), 0], [k/np.sqrt(2), k*np.sqrt(2)], 'grey')
    plt.plot([0,k/np.sqrt(2)], [k*np.sqrt(2),k/np.sqrt(2)], 'grey')

    plt.hist2d(ws,vs, bins=[bin_size, bin_size]);
    plt.grid(False)
    plt.ylim([0, k*np.sqrt(2)])
    plt.xlim([-k/np.sqrt(2), k/np.sqrt(2)])
    plt.set_cmap(plt.cm.get_cmap('Greys'))  #Purples, YlOrRd, GnBu，Greys
    plt.title(title)
    

In [ ]:
# Setting:  phi = 0.1, num_trials = 50000

bin_size = 100
plotting(w_array[1:], v_array[1:], bin_size, title='Magnet Rule')
plotting(w_array_alt[1:],v_array_alt[1:], bin_size, title='Straight Ahead Rule')
plotting(w_array_two_opt[1:-1],v_array_two_opt[1:-1], bin_size, title='Two-Opt Rule')
plotting(w_array_three_opt[1:-2],v_array_three_opt[1:-2], bin_size, title='Three-Opt Rule')

## Toronto: Given walking radius, lamda, and rho, simulate to get waiting time 

In [ ]:
solution = Table.read_table('solution_toronto.csv')
zipcode_pop_speed_toronto =  Table.read_table('zipcode_pop_speed_toronto.csv')

# add speed info
solution2 = solution.join('zipcode', zipcode_pop_speed_toronto, 'FSA').drop(25,26,27,28,29,30,31).sort('ID')
solution2

## FSAs in the Within-Radius Mode

In [ ]:
solution_mg1 = solution2.where('mode_opt_array',0)
solution_mg1.show()

In [ ]:
solution_mg1 = solution2

In [ ]:
num_trials = 50000
num_FSA_mg1 = solution_mg1.num_rows
mu = 60 / 5  # rate of serving a customer
#mu = 60/5/2
service_intervals = 1/mu * np.ones(num_trials)
scale_factor = 100

Simulated_mean_wait = np.zeros(num_FSA_mg1)
Theory_mean_wait = np.zeros(num_FSA_mg1)
Simulation_error = np.zeros(num_FSA_mg1)

for row in range(num_FSA_mg1):
#for row in [0]:

    print( '\n', 'FSA ID: ', solution_mg1.column('ID')[row], '\n')

    rho_opt = solution_mg1.column('rho_mg1_array')[row]
    R_opt = 1/np.sqrt(rho_opt)
    r_opt = solution_mg1.column('radius_mg1_array')[row]
    lamb = solution_mg1.column('lamb_endo_mg1_array')[row]/rho_opt
    
    speed = solution_mg1.column('speed')[row]
    #speed = 10
    
    order_intervals = np.random.exponential(1/lamb, num_trials)
    order_times = np.cumsum(order_intervals)
    
    
    #=============== simulate stall movement ==========
    R = round(R_opt, 2) * scale_factor  # scale it to be an integer of right order of magnitude
    k=int(R)

    r = r_opt * scale_factor
    phi = r/np.sqrt(2)/R    

    #print('phi: ', phi, '\n')

    x_c = R/2*np.ones(num_trials)
    y_c = R/2*np.ones(num_trials)

    x_array = R/2*np.ones(num_trials)
    y_array = R/2*np.ones(num_trials)
    
    service_start_times = -55*np.ones(num_trials)
    wait_actual_intervals = -55*np.ones(num_trials)
    

    for i in range(num_trials):
            #  customer's location
        x_c[i], y_c[i] = generate_customer_pos(k)

    for i in range(num_trials-1):    
        x_array[i+1], y_array[i+1] = magnet_rule(x_array[i], y_array[i], x_c[i+1], y_c[i+1] )
        
    
        # change the coordinate from x-y to v-w to be upright:
    w_array = x_array/np.sqrt(2) - y_array/np.sqrt(2)
    v_array = x_array/np.sqrt(2) + y_array/np.sqrt(2)

    repo_dist_array = np.abs(w_array[1:]-w_array[:-1]) + np.abs(v_array[1:]-v_array[:-1]) 
    repo_interval_array = repo_dist_array/speed/scale_factor
    
    
    service_start_times[0] = order_times[0]    
    for i in range(num_trials-1):
        # incorrect formula
        #arrival_times[i+1] = max(order_times[i+1], arrival_times[i]+service_intervals[i])+repo_interval_array[i]
        
        # correct formula
        service_start_times[i+1] = max(order_times[i+1],  service_start_times[i]+repo_interval_array[i-1]+service_intervals[i])


    wait_intervals = service_start_times-order_times
    wait_actual_intervals[0]=0
    wait_actual_intervals[1:] = wait_intervals[1:] + repo_interval_array
    
    Simulated_mean_wait[row] = np.mean(wait_actual_intervals[1:])
    Theory_mean_wait[row] = solution_mg1.column('wait_endo_mg1_array')[row]
    Simulation_error[row] = abs(Simulated_mean_wait[row]-Theory_mean_wait[row]) / Theory_mean_wait[row]
    
    print(' Simulated mean wait: ', Simulated_mean_wait[row], '\n', 
          'Theoretical mean wait: ', Theory_mean_wait[row],'\n',
         'Simulation error: ', Simulation_error[row], '\n')

print('Average Simulation Error: ', np.mean(Simulation_error) )

## FSAs in the Pooling Mode

In [ ]:
solution_pool = solution2.where('mode_opt_array',1)
solution_pool

In [ ]:
import copy

In [ ]:
# generate randomness
num_trials = 50000
order_intervals_fixed = np.random.exponential(1/lamb, num_trials)
customer_location_fixed = np.random.choice(N,num_trials)  # randomly assign customers to N sub-zones


In [ ]:
solution_pool = solution2

In [ ]:
num_FSA_pool = solution_pool.num_rows
mu = 60 / 5  # rate of serving a customer
#mu = 60/5/2
service_intervals = 1/mu * np.ones(num_trials)
scale_factor = 100   


Simulated_mean_wait = np.zeros(num_FSA_pool)
Theory_mean_wait = np.zeros(num_FSA_pool)
Simulation_error = np.zeros(num_FSA_pool)


for row in range(num_FSA_pool):
#for row in [0]:

    print( '\n', 'FSA ID: ', solution_pool.column('ID')[row], '\n')

    rho_opt = solution_pool.column('rho_pooling_array')[row]
    R_opt = 1/np.sqrt(rho_opt)
    r_opt = solution_pool.column('radius_pooling_array')[row]
    lamb = solution_pool.column('lamb_endo_pooling_array')[row]/rho_opt
    tp = solution_pool.column('t_pooling_array')[row]

    
    speed = solution_pool.column('speed')[row]
    #speed = 10
    
    order_intervals = order_intervals_fixed
    order_times = np.cumsum(order_intervals)
    wait_intervals = -55*np.ones(num_trials)
    
    #=============== simulate stall movement ==========
    R = round(R_opt, 2) * scale_factor  # scale it to be an integer of right order of magnitude
    k=int(R)

    r = r_opt * scale_factor
    
    N_unrounded = (R/(np.sqrt(2)*r))**2 # number of sub-zones
    N = round(N_unrounded)
    print('N_unrounded: ', N_unrounded)
    r_adjusted = np.sqrt(R**2/N)/np.sqrt(2)
    customer_location = customer_location_fixed  # randomly assign customers to N sub-zones

    stall_location_now = 0
    service_start_times[0] = order_times[0]    
    
    flag = 'ON'
    i=0
    time_now = 0
    customer_now = 0
    wait_list = []
    
    count = 0

    wait_list_now = []
    
    while flag == 'ON':
                                
        wait_list_temp = copy.copy(wait_list)
        for old_customer in wait_list_temp:
            if customer_location[old_customer] == stall_location_now:  # finally arrives
                wait_intervals[old_customer] = time_now - order_times[old_customer]
                wait_list.remove(old_customer)
                
        
        if order_times[customer_now] > time_now + tp:   # no customer orders at all during the stay 
            time_leaving = time_now + tp
            # process customers that arrive during the time window between stall departure from the current sub-zone and arrival at the next sub-zone
            while order_times[customer_now] < time_leaving + 2*r_adjusted/speed/scale_factor:
                wait_list.append(customer_now)
                customer_now = customer_now + 1
                if customer_now >= num_trials:
                    flag = 'OFF'
                    break 
            
        else:
            # process customers that arrive during the promised time window: either serve, or waitlist
            while order_times[customer_now] <= time_now + tp: 
                
                if order_times[customer_now] < time_now:
                    print('Error')
                    count = count+1
                last_customer_in_the_subzone = 'NA'
                
                if customer_location[customer_now] == stall_location_now:  # happen to be in the same sub-zone
                    wait_intervals[customer_now] = 0
                    last_customer_in_the_subzone = customer_now
                    
                else:    # in a different sub-zone
                    wait_list.append(customer_now) 
                    #print(wait_list)

                customer_now = customer_now + 1
                
                if customer_now >= num_trials:
                    flag = 'OFF'
                    break 
                    
            if last_customer_in_the_subzone == 'NA':  # all arriving orders are from other subzones
                time_leaving = time_now + tp
            else:
                time_leaving = max(time_now + tp, order_times[last_customer_in_the_subzone] + service_intervals[last_customer_in_the_subzone])
            
            # process customers that arrive during the time window between stall departure from the current sub-zone and arrival at the next sub-zone
            if customer_now < num_trials: 
                while order_times[customer_now] < time_leaving + 2*r_adjusted/speed/scale_factor:
                    wait_list.append(customer_now) 
                    
                    customer_now = customer_now + 1
                    
                    if customer_now >= num_trials:
                        flag = 'OFF'
                        break 
            
        
        time_now = time_leaving + 2*r_adjusted/speed/scale_factor
        stall_location_now = (stall_location_now+1)%N
        #print(stall_location_now)      
    
    Simulated_mean_wait[row] = np.mean(wait_intervals[:int(0.9*num_trials)])
    Theory_mean_wait[row] = solution_pool.column('wait_endo_pooling_array')[row]
    Simulation_error[row] = abs(Simulated_mean_wait[row]-Theory_mean_wait[row]) / Theory_mean_wait[row]
    
    print(' Simulated mean wait: ', Simulated_mean_wait[row], '\n', 
          'Theoretical mean wait: ', Theory_mean_wait[row],'\n',
         'Simulation error: ', Simulation_error[row], '\n')

print('Average Simulation Error: ', np.mean(Simulation_error) )